# Generating Random Numbers 
### Author: Max Al-Hasso

Welcome! This blog is the first entry in the series *An Introduction to Quantum Cryptography*. In this post, and throughout the series, we are going to investigate how quantum computers can be used to enable secure communication and to understand the quantum algorithms that allow us to do so. To start with, we are going to look at the task of generating randomness - an essential part of creating a secure cryptographic system.

Prerequisites:
- Complex numbers
- Dirac notation
- Vector spaces

Topics covered:
- Qubits
- Bases of quantum states
- The Hadamard Gate
- Quantum measurement

## Classical methods for generating random numbers

If you were tasked with generating a random number, there are a lot of ways you might go about trying to do so. Perhaps you roll some dice or draw a card. Or perhaps you measure something in the environment, such as Brownian motion and report your findings. Okay, so far, so good. The next task, however, is to create a computer program that outputs a random number each time it is run. The problem here is that computers are inherently deterministic, meaning if I run the same program (with the same parameters) twice I should expect the same result each time. So how do we bend the rules of these deterministic machines to perform our task of generating random numbers? The answer comes in the form of a type of algorithm called a pseudorandom number generator (PRNGs).

A PRNG is a deterministic algorithm that spits out numbers seemingly at "random". In fact, there are incredibly good PRNGs called Cryptographically Secure PRNGs that are used today in cryptography. However, there are two things which make a PRNG only pseudorandom, not *truly* random. First, a PRNG requires a *seed* from which the random string is generated. Using the same seed twice will result in the same string of random numbers (as the PRNG is a deterministic algorithm). Second, it is very hard to prove that, when the PRNG is run for a very long time, the numbers remain random. Therefore, when using a PRNG (for cryptographic purposes) the seeds should be periodically refreshed.

This may seem circular, as now it seems we need to periodically generate random numbers as seeds for the PRNG. The key here is to make sure an adversary (someone trying to break your cryptographic system) cannot learn the seed that you choose as they would then be able to generate the same string of random numbers and break your system. There are many different methods for generating seeds, your laptop might measure the temperature of it's CPU to a fine precision and use that as a seed. Alternatively, the company Cloudflare takes photos of a wall of lava lamps and uses the binary representation as seeds in their cybersecurity services.
![[lava-lamps.webp]]

## A Quantum Random Number Generator
As far as we know, quantum processes can exhibit *truly* random behavior. So, if we had a quantum computer, is it possible to generate truly random numbers from a deterministic algorithm? The answer to this question is yes! Let's take a look at how.

If you're reading this you are likely familiar with the concept of a qubit, a unit of information used in quantum computers that can be in a superposition of both the 0 and 1 state simultaneously. More formally, a qubit is a 2-dimensional complex vector that can be represented as $$\ket{\psi}=\alpha\ket{0}+\beta\ket{1}, \quad \text{where } \alpha, \beta \in \mathbb{C} \text{ and }|\alpha|^2+|\beta|^2=1$$
As a quick recap, $\ket{\cdot}$ is called a *ket* and is a complex column vector. The quantum state of the qubit $\ket{\psi}$ is therefore a 2-dimensional column vector with complex elements.
### A quick aside on vector spaces
Recall that a vector space is generated by the linear combination of a set of basis vectors. For example, a 2-dimensional *real* vector space has the basis vectors $\left\{(1,0), (0,1)\right\}$. Any point in this 2-dimensional plane can be reached using some combination of these two vectors. E.g. $$(10, 5)=10\cdot(1,0)+5\cdot(0,1)$$Equally, the same plane ($\mathbb{R}^2$) also has the basis $\left\{(1,1), (1,-1)\right\}$. E.g.$$(10, 5)=\frac{15}{2}\cdot(1,1)+\frac{5}{2}\cdot(1,-1)$$
In fact, there are infinitely many bases for a vector space. In all of these basis, any point can be described by the linear combination of the basis vectors with some real coefficients.

For the state space of a qubit, which is $\mathbb{C}^2$, there is some nuance to consider. This is a *complex* vector space. This means that the coefficients in the linear combination are now complex numbers. The *standard basis*, also called the *computational basis*, for $\mathbb{C}^2$ is $\{\ket{0}, \ket{1}\}$ where $$\ket{0}=\begin{bmatrix}1 \\ 0\end{bmatrix},\quad\ket{1}=\begin{bmatrix}0 \\ 1\end{bmatrix}.$$This brings us back to our definition of a qubit, $$\ket{\psi}=\alpha\ket{0}+\beta\ket{1}, \quad \text{where } \alpha, \beta \in \mathbb{C} \text{ and }|\alpha|^2+|\beta|^2=1$$We can see that $\ket{\psi}$ is just a vector in the *complex* vector space generated by the standard basis, subject to the condition that $|\alpha|^2+|\beta|^2=1$. As before, there are infinitely many bases for the state space of a qubit, and we will see one more shortly.

### Hadamard Gate
For our randomness-generating algorithm, we are going to need the faithful Hadamard gate. If your just meeting the Hadamard gate for the first time, this guy will soon become a fast friend on your journey into quantum computing. The Hadamard gate is defined as $H=\frac{1}{\sqrt{2}}\begin{bmatrix}1 & 1 \\1 & -1\end{bmatrix}$.

Recall that gates are *unitary* matrices that can be applied to quantum states to change the state. Lets see an important pair of examples, the states $\ket{+}$ and $\ket{-}$:

$H\ket{0}=\frac{1}{\sqrt{2}}\begin{bmatrix}1 & 1 \\1 & -1\end{bmatrix}\begin{bmatrix}1 \\ 0\end{bmatrix}=\frac{1}{\sqrt{2}}\begin{bmatrix}1 \\ 1\end{bmatrix}=\ket{+}$

$H\ket{1}=\frac{1}{\sqrt{2}}\begin{bmatrix}1 & 1 \\1 & -1\end{bmatrix}\begin{bmatrix}0 \\ 1\end{bmatrix}=\frac{1}{\sqrt{2}}\begin{bmatrix}1 \\ -1\end{bmatrix}=\ket{-}$

This is an example of quantum superposition! The states $\ket{+}$ and $\ket{-}$ are each partially in state $\ket{0}$ and $\ket{1}$ at the same time. We can tell by looking at the vector representation of the states and seeing that both elements are non-zero. Furthermore, $\{\ket{+}, \ket{-}\}$ form another basis of $\mathbb{C}^2$, the qubit state space.

### Measuring qubits
For the final component of our algorithm, we are going to need to be able to measure qubits and be able to calculate the probability of obtaining a certain measurement. 

Say I have a qubit $\ket{\psi}$ that I want to measure. The first thing I need to do is specify which basis I want my measurement in. We are familiar with the standard basis, and that seems like a good choice, so let's use that for now.

To calculate the probability of a quantum state $\ket{\psi}$ collapsing to a given state $\ket{\phi}$ when measured, we use the following equation $P_{\phi}=|\braket{\psi|\phi}|^2$ where $\braket{\psi|\phi}$ is the inner product of $\ket{\psi}$ and $\ket{\phi}$.
### Putting it all together
Okay, we are ready to create our randomness-generating quantum algorithm. We start with a qubit in the $\ket{0}$ state, apply a single Hadamard gate, and then measure the qubit in the standard basis. That's it!

In [ ]:
# This code draws our randomness-generating quantum circuit

from qiskit import QuantumCircuit

qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)
qc.draw(output='text')

┌───┐┌─┐
  q: ┤ H ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0

In [ ]:
# This code uses Qiskit to simulate our circuit

from qiskit import transpile
from qiskit_aer import AerSimulator

backend = AerSimulator()
qc_compiled = transpile(qc, backend)
job_sim = backend.run(qc_compiled, shots=1024)
result_sim = job_sim.result()
counts = result_sim.get_counts(qc_compiled)
print(counts)


{'1': 503, '0': 521}


Let's see why this works:
We know from before that $H\ket{0} = \ket{+}$. So what is the probability of measuring each of $\ket{0}$ and $\ket{1}$ when $\ket{+}$ is measured in the standard basis? We can use the formula from the previous section.

$P_{0}=|\braket{+|0}|^2=\left|\frac{1}{\sqrt{2}}\begin{bmatrix}1 \\ 1\end{bmatrix} \cdot \begin{bmatrix}1 \\ 0\end{bmatrix}\right|^2=\frac{1}{2}$

$P_{1}=|\braket{+|1}|^2=\left|\frac{1}{\sqrt{2}}\begin{bmatrix}1 \\ 1\end{bmatrix} \cdot \begin{bmatrix}0 \\ 1\end{bmatrix}\right|^2=\frac{1}{2}$

There we go, with just a single qubit and the application of one quantum gate we have created a *deterministic* program that generates a truly random number. What is especially interesting about this program is that, as far as we know, no matter how powerful our advisory is there is absolutely no way that they could improve on a 50:50 guess - even if they could perfectly calculate how a die would roll or predict the movements of Cloudfare’s lava lamps with infinite precision.


---
This blog post, and the others in this series are adapted from *Introduction to Quantum Cryptography* by Thomas Vidick and Stephanie Wehner, if you are interested in exploring these topics further I would highly recommend this textbook.
